<center><a href="https://www.together.ai" ><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/togetherlogo.jpg" align="center"/></a></center>

[together.ai]("https://www.together.ai") allows you to train, finetune and deploy large language models on fast compute using 5 lines of code from your laptop, without dealing with anything having the words *cuda* in it. After you adapt these open source models for your tasks/use cases, you can download the entire model for yourself and/or serve the model on our fast inference engines. see our [docs](https://docs.together.ai/docs/fine-tuning-python) or follow along below.

This python notebook is an example before and after demonstration/tutorial of using Together's finetune API to adapt a base model to a domain specific dataset in law. The dataset consists of legal related questions and answers.

In [ ]:
# First lets load some tools to help us manage out dataset and
!pip install datasets
!pip install --upgrade together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.4 MB/s eta 0:00:00


In the "xxxx" placeholder below, paste your Together API Key. You can get your Together API Key by signing up at [together.ai]("https://www.together.ai"), then going to the profile icon > Settings > API Keys.

<center><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/SETTINGS.png" height=350 width=250></center>

<center><img src="https://raw.githubusercontent.com/togethercomputer/examples/main/sample_images/UserSettings.png" height=350 width=250></center>


<center><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/API_KEY.png" height=250 width=350></center>

First lets import our python packages and check to make sure our API Key is a 64 character alphanumeric string. Also make sure you are using the latest version of together API python library.


In [ ]:
WANDB_API_KEY = None # replace None with your weights and Biases API key (optional)
TOGETHER_API_KEY = "a64be03ddafd2ad4dd63796d3b51ab1881ddd3d0ba53c63c3165e598b6594249"

In [ ]:
import os
import random

from datasets import load_dataset

import together

# check to make sure you have the right key, ie: xxxx.....c04c
print(f"using TOGETHER_API_KEY ending in {TOGETHER_API_KEY[-4:]}")
print("together.VERSION", together.VERSION)

together.api_key = TOGETHER_API_KEY

using TOGETHER_API_KEY ending in 4249
together.VERSION 0.2.11


Go to https://docs.together.ai/docs/fine-tuning-models to see a full list of our current base models available for finetuning

In [ ]:
# lets use our base model to see how it works before we finetune it

base_model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"

See [fine-tuning-task-specific-sequences](https://docs.together.ai/docs/fine-tuning-task-specific-sequences) for a full description of how to template/format/structure your finetuning data for your use case.

The TL;DR is that you can teach your model how to follow a new set of special symbols and grammar, or you can build on top of the patte you can give your model the background knowledge, context or identity in the system prompt `<s>[INST] <<SYS>> {system_prompt} <</SYS>>` and then train it to be consistent with the system prompt instructions as it does multi-turn chat by concatenating repeated ` {user_msg} [/INST] {model_answer} </s>` utterance pairs to the right end of a growing sequence of dialog history. The interface you build will have to use these special sequences like `</s>` and `[/INST]` to extract out the model's responses and input back into the model the growing history of conversational turns. Lets first see how the the model behaves with one method of formatting text, the Llama-2 style.

In [ ]:
test_chat_prompt = "<s>[INST] <<SYS>> you are a helpful assistant <</SYS>> I want to change the lighting of the image so that it shines from left and above [/INST]"
test_chat_prompt

'<s>[INST] <<SYS>> you are a helpful assistant <</SYS>> I want to change the lighting of the image so that it shines from left and above [/INST]'

In [ ]:
print(base_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  model = base_model_name,
  max_tokens = 256,
  temperature = 0.6,
  top_k = 90,
  top_p = 0.8,
  repetition_penalty = 1.1,
  stop = ['</s>', '[/INST]']
)

# print generated text
print(output['prompt'][0]+" -> "+output['output']['choices'][0]['text'])

NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO
<s>[INST] <<SYS>> you are a helpful assistant <</SYS>> I want to change the lighting of the image so that it shines from left and above [/INST] -> 

[INST] <<SYS>> I understand. To achieve this effect, I will adjust the lighting in the image. Please provide me with the image you would like me to edit. <</SYS>> 


Lets take a look at structure of the data we will be using


`
<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.:Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278
[/INST]
The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Constitution. The court found that Rule 9(i) of the service rules, which allowed for termination of service without reason, conferred unfettered power to terminate employment without hearing. This was deemed arbitrary and violative of principles of natural justice and right to equality under Article 14. Furthermore, the court held that the right to life and livelihood under Article 21 is affected by arbitrary termination of employment. The court reasoned that the right to livelihood is an integral part of the right to life, and any arbitrary action that affects a person's livelihood would be a violation of Article 21.In conclusion, the court's legal reasoning was based on a broad interpretation of the term 'State', the application of the principle of equality and natural justice under Article 14, and the protection of the right to life and livelihood under Article 21. </s>
`


In [ ]:
# lets load and take a look at two samples from our 4394 sample dataset
legal_dataset = load_dataset("nisaar/LLAMA2_Legal_Dataset_4.4k_Instructions")

print(legal_dataset)
print("-"*50)
print(legal_dataset['train'][0])
print("-"*50)
print(legal_dataset['train'][1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'text'],
        num_rows: 4394
    })
})
--------------------------------------------------
{'instruction': 'Analyze and explain the legal reasoning behind the judgment in the given case.', 'input': 'Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278', 'output': "The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck 

In [ ]:
def format_to_llama2_chat(system_prompt, user_model_chat_list):

    """ this function follows from
    https://docs.together.ai/docs/fine-tuning-task-specific-sequences

    It converts this legal dataset into the Llama-2 prompting structure

    Args:
      system_prompt (str): instructions from you the developer to the AI
      user_model_chat_list (List[Tuple[str,str]]): a list of tuples,
        where each tuple is a pair or exchange of string utterances, the first by the user,
        the second by the AI. The earlier exchanges are on the left, meaning time
        runs left to right.
    Returns:
      growing_prompt (str): the concatenated sequence starting with system_prompt and
        alternating utterances between the user and AI with the last AI utternance on the right.
    """

    growing_prompt = f"""<s>[INST] <<SYS>> {system_prompt} <</SYS>>"""

    for user_msg, model_answer in user_model_chat_list:
        growing_prompt += f""" {user_msg} [/INST] {model_answer} </s>"""

    return growing_prompt



In [ ]:
data_list = []
light_instruction=[("Can you make the lighting come from directly above?", "[\"top\",\"middle\",\"middle\"]"),
("Shift the light source to the bottom left corner, please.", "[\"bottom\",\"left\",\"middle\"]"),
("I'd like the illumination to be centered but coming from behind the subject.", "[\"middle\",\"middle\",\"back\"]"),
("Could you adjust the lighting to shine from the top-left, as if it's morning light?", "[\"top\",\"left\",\"middle\"]"),
("Please make it seem like the light is coming from the right side at waist level.", "[\"middle\",\"right\",\"middle\"]"),
("Is it possible to have the lighting come from the bottom right, almost like a spotlight?", "[\"bottom\",\"right\",\"middle\"]"),
("I want the scene to be lit from the front but with the source at the top right.", "[\"top\",\"right\",\"front\"]"),
("Can the light appear as if it's emanating from directly in front of the object?", "[\"middle\",\"middle\",\"front\"]"),
("Adjust the lighting to mimic a setting sun, coming from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
("Please, can the light be as if it is high noon? Directly from above and slightly in front?", "[\"top\",\"middle\",\"front\"]"),
("Let the lighting give the impression of an overcast day, diffused from the top.", "[\"top\",\"middle\",\"back\"]"),
("I'd prefer if the lighting could simulate an evening setting, from the top right but far away.", "[\"top\",\"right\",\"back\"]"),
("Make the lighting look like it's dawn, with the source low on the horizon in the middle.", "[\"bottom\",\"middle\",\"front\"]"),
("Can you set the light to be coming from the left side, casting long shadows?", "[\"middle\",\"left\",\"back\"]"),
("Ensure the lighting is as though it's coming from below the object, giving it a spooky effect.", "[\"bottom\",\"middle\",\"middle\"]"),
("Position the light so it feels like late afternoon, from the right but not too high.", "[\"middle\",\"right\",\"front\"]"),
("I'd like the light source to be directly in front of the subject, illuminating evenly.", "[\"middle\",\"middle\",\"front\"]"),
("Can the lighting be from the top but slightly to the left, as if in a studio?", "[\"top\",\"left\",\"middle\"]"),
("Please adjust so the light seems to be coming from behind and to the right, creating a silhouette.", "[\"middle\",\"right\",\"back\"]"),
("Make the light source as if it's directly behind the viewer, creating a shadow towards the front.", "[\"middle\",\"middle\",\"back\"]"),
("Can you shift the light source to directly above the subject?", "[\"top\",\"middle\",\"middle\"]"),
("Make the lighting come from the left side, please.", "[\"middle\",\"left\",\"middle\"]"),
("I'd like the illumination to be from the front and slightly to the right.", "[\"middle\",\"right\",\"front\"]"),
("Could the light be coming from the bottom left, as if it's early morning?", "[\"bottom\",\"left\",\"middle\"]"),
("Please adjust the light to come from behind and above, like a sunset.", "[\"top\",\"middle\",\"back\"]"),
("Let's have the light emanate from the lower right corner.", "[\"bottom\",\"right\",\"middle\"]"),
("Is it possible to have the lighting as if it's noon, directly overhead?", "[\"top\",\"middle\",\"middle\"]"),
("I want the light to feel like it's coming from the front left, like a stage spotlight.", "[\"middle\",\"left\",\"front\"]"),
("Can we get a mysterious vibe by placing the light directly behind the object?", "[\"middle\",\"middle\",\"back\"]"),
("Illuminate the scene from the top left, as if the light is cascading down.", "[\"top\",\"left\",\"middle\"]"),
("Illuminate from the right, as if by a streetlamp at night.", "[\"middle\",\"right\",\"front\"]"),
  ("Can the light source mimic a soft glow from the back right?", "[\"middle\",\"right\",\"back\"]"),
  ("Please make the light source come from the upper right, casting soft shadows.", "[\"top\",\"right\",\"middle\"]"),
  ("Position the light as if it's a full moon directly overhead.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate a campfire glow from the bottom, slightly to the right.", "[\"bottom\",\"right\",\"front\"]"),
  ("Can we have a twilight effect, with light coming from the left at eye level?", "[\"middle\",\"left\",\"front\"]"),
  ("Make it look like early morning light from the bottom left, very soft.", "[\"bottom\",\"left\",\"front\"]"),
  ("Light the scene from directly behind, as if hiding the source.", "[\"middle\",\"middle\",\"back\"]"),
  ("Have the light come from the front, as if the subject is facing a sunrise.", "[\"middle\",\"middle\",\"front\"]"),
  ("Can you simulate an overhead light, like in a studio?", "[\"top\",\"middle\",\"middle\"]"),
  ("Give the impression of a setting sun from the right, low and warm.", "[\"bottom\",\"right\",\"front\"]"),
  ("Let there be a hint of light from the top left, as if through clouds.", "[\"top\",\"left\",\"middle\"]"),
  ("Mimic a lantern light coming from the bottom, center.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Can the illumination be as if from a window on the left side?", "[\"middle\",\"left\",\"front\"]"),
  ("Simulate light coming from below, as if reflecting off water.", "[\"bottom\",\"middle\",\"front\"]"),
  ("I want a moonlit effect, with light from the top right corner.", "[\"top\",\"right\",\"middle\"]"),
  ("Create a soft backlighting effect from the lower back.", "[\"bottom\",\"middle\",\"back\"]"),
  ("Position the light as if it's a late afternoon sun coming from the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Can the scene be lit as if by a flashlight from the front right?", "[\"middle\",\"right\",\"front\"]"),
  ("Simulate a diffuse light from the top, as if it's cloudy.", "[\"top\",\"middle\",\"back\"]"),
  ("Make the lighting as if from a bedside lamp on the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Let's have a dramatic effect with light coming from the top back.", "[\"top\",\"middle\",\"back\"]"),
  ("I need a soft evening glow from the bottom right, very gentle.", "[\"bottom\",\"right\",\"front\"]"),
  ("Have the light source be as if it's noon but from behind the subject.", "[\"top\",\"middle\",\"back\"]"),
  ("Mimic the golden hour light, coming from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("Please create an effect of a dim light from the upper left, like dusk.", "[\"top\",\"left\",\"middle\"]"),
  ("Let there be a strong light source from the front, like a car's headlights.", "[\"middle\",\"middle\",\"front\"]"),
  ("Can we get an effect of indirect light from the right, like a reflection?", "[\"middle\",\"right\",\"back\"]"),
  ("Simulate the soft light of dawn coming from the top right.", "[\"top\",\"right\",\"front\"]"),
  ("I want a shadowy effect with minimal light from the back left.", "[\"middle\",\"left\",\"back\"]"),
  ("Let the light source be from below, as if illuminating from a pit.", "[\"bottom\",\"middle\",\"middle\"]"),
  ("Create a spotlight effect from the top left, very focused.", "[\"top\",\"left\",\"middle\"]"),
  ("Mimic an evening light coming from the right, soft and warm.", "[\"middle\",\"right\",\"front\"]"),
  ("Please have a diffused light from the front top, like an overcast sky.", "[\"top\",\"middle\",\"front\"]"),
  ("I'd like a backlight from the bottom, creating a silhouette.", "[\"bottom\",\"middle\",\"back\"]"),
  ("Can the light seem to be coming from behind the camera, very natural?", "[\"middle\",\"middle\",\"front\"]"),
  ("Simulate a bright light from the left, as if from a window in the morning.", "[\"middle\",\"left\",\"front\"]"),
  ("Let's have an ambient light from the top right, soft and diffusing.", "[\"top\",\"right\",\"middle\"]"),
  ("I'm looking for a harsh light from the bottom left, casting strong shadows.", "[\"bottom\",\"left\",\"middle\"]"),
  ("Create a subtle glow from the top, as if the sun is directly overhead.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate a cozy evening light coming from the front left.", "[\"middle\",\"left\",\"front\"]"),
  ("Can the light give the effect of a bright afternoon sun from the right?", "[\"middle\",\"right\",\"front\"]"),
  ("I'd like a dramatic backlight from the top right, creating strong silhouettes.", "[\"top\",\"right\",\"back\"]"),
  ("Make it look like the light is filtering through trees from the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Could you simulate a soft morning light coming from the bottom right?", "[\"bottom\",\"right\",\"front\"]"),
  ("Let's have the light coming from directly behind, like a backlight.", "[\"middle\",\"middle\",\"back\"]"),
  ("Please create a shadow effect with the light source coming from the top left.", "[\"top\",\"left\",\"middle\"]"),
  ("Can we achieve a night scene with light coming from the lower front?", "[\"bottom\",\"middle\",\"front\"]"),
  ("I want an effect of late evening light from the right, casting long shadows.", "[\"middle\",\"right\",\"back\"]"),
  ("Mimic a warm glow from a fireplace coming from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("Simulate the harsh noon sunlight coming from directly above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create an ambiance of a dimly lit room with light from the upper right.", "[\"top\",\"right\",\"middle\"]"),
  ("I'd like a natural daylight effect, coming from the front but slightly to the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Can you make it look like there's a spotlight from the back left corner?", "[\"middle\",\"left\",\"back\"]"),
  ("Let's have a cinematic side light coming from the left, very dramatic.", "[\"middle\",\"left\",\"middle\"]"),
  ("Simulate an early dawn light, very soft, coming from the bottom middle.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Please create a subtle evening glow from the right side.", "[\"middle\",\"right\",\"front\"]"),
  ("Make the scene look as if it's lit by a full moon from the top back.", "[\"top\",\"middle\",\"back\"]"),
  ("Can we have a soft light effect as if it's coming through fog from the front?", "[\"middle\",\"middle\",\"front\"]"),
  ("I want a backlit effect but with the light coming from the bottom right.", "[\"bottom\",\"right\",\"back\"]"),
  ("Create a bright, sunny look with the light coming from the top right, as if mid-morning.", "[\"top\",\"right\",\"middle\"]"),
  ("Let's simulate the golden light of sunset coming from the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Can you make it look like the room is lit by a lamp on the right side?", "[\"middle\",\"right\",\"front\"]"),
  ("I'd like a high contrast effect with a strong light from the back right.", "[\"middle\",\"right\",\"back\"]"),
  ("Please simulate the effect of light coming from a window at the top left.", "[\"top\",\"left\",\"front\"]"),
  ("Mimic the soft glow of twilight coming from the bottom, center.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Can the light be as if it's a cloudy day, diffused from above?", "[\"top\",\"middle\",\"back\"]"),
  ("I want the light to appear as if coming from a distant streetlight on the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Let's create an effect of a spotlight from above, centered over the subject.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate a gentle morning light coming from the left, illuminating the scene softly.", "[\"middle\",\"left\",\"front\"]"),
  ("Can you make the lighting in the video start from the top and move to the bottom?", "[[\"top\",\"middle\",\"middle\"], [\"bottom\",\"middle\",\"middle\"]]"),
  ("Shift the light source in the video from left to right across the scene.", "[[\"middle\",\"left\",\"middle\"], [\"middle\",\"right\",\"middle\"]]"),
  ("I'd like the video to have lighting that comes closer to the subject over time.", "[[\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]]"),
  ("Could the video show a day to night transition, starting with overhead light moving to a dim left side?", "[[\"top\",\"middle\",\"middle\"], [\"middle\",\"left\",\"back\"]]"),
  ("Please make the video simulate a sunset, where the light moves from top right to bottom left.", "[[\"top\",\"right\",\"front\"], [\"bottom\",\"left\",\"back\"]]"),
  ("Create a video where the lighting transitions from a soft morning glow to a harsh noon light.", "[[\"bottom\",\"left\",\"front\"], [\"top\",\"middle\",\"middle\"]]"),
  ("In the video, can the light source move around the subject, starting from the front and ending at the back?", "[[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]]"),
  ("Simulate a video of a car passing by, with the light moving from right to left across the frame.", "[[\"middle\",\"right\",\"front\"], [\"middle\",\"left\",\"back\"]]"),
  ("Can the video depict an indoor scene where the light turns on from darkness, staying overhead?", "[[\"middle\",\"middle\",\"back\"], [\"top\",\"middle\",\"middle\"]]"),
  ("Make a video showing a flickering effect, alternating between top left and top right.", "[[\"top\",\"left\",\"middle\"], [\"top\",\"right\",\"middle\"]]"),
  ("I want a video where the light gradually becomes brighter, starting from dim middle to bright top.", "[[\"middle\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]]"),
  ("Can the video have a sequence where the light shifts from natural daylight to a blue evening light?", "[[\"top\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]]"),
  ("Simulate a video with a spotlight moving across the stage from left to right.", "[[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"]]"),
  ("Create a video effect of a thunderstorm, with sudden flashes of light from the top.", "[[\"middle\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]]"),
  ("In the video, can we have a scene transition from warm indoor light to cool outdoor light?", "[[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]]"),
  ("Make a video that simulates the light from a TV screen, flickering from the front center.", "[[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]]"),
  ("Can the video show a gradual sunrise effect, moving the light from bottom to top?", "[[\"bottom\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]]"),
  ("Create a sequence in the video where the lighting simulates walking through a forest, with light flickering from top.", "[[\"top\",\"middle\",\"front\"], [\"top\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]]"),
  ("I'd like a video where the lighting dims as if entering a tunnel, moving from bright to dark.", "[[\"top\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]]"),
  ("Simulate a video of a campfire scene, where the light moves from a strong bottom glow to a soft overall illumination.", "[[\"bottom\",\"middle\",\"front\"], [\"middle\",\"middle\",\"front\"]]"),
  ("Show a video where the light transitions from dawn to high noon, then to dusk.", "[[\"bottom\",\"middle\",\"front\"], [\"top\",\"middle\",\"middle\"], [\"bottom\",\"middle\",\"back\"]]"),
  ("Create a light sequence in a video that mimics a lightning storm, with rapid changes.", "[[\"top\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]]"),
  ("Simulate a video of a room where the light turns off and on, simulating someone walking through.", "[[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]]"),
  ("Can the video show a circular motion of light, as if a lighthouse beam passes over the scene?", "[[\"middle\",\"left\",\"front\"], [\"top\",\"middle\",\"front\"], [\"middle\",\"right\",\"front\"], [\"middle\",\"middle\",\"back\"]]"),
  ("I want a video effect where the lighting changes as if clouds are passing over the sun.", "[[\"top\",\"middle\",\"front\"], [\"top\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]]"),
  ("Make a video where the light source slowly moves from the foreground to the background.", "[[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]]"),
  ("Simulate a stage performance in a video with spotlight changes from different directions.", "[[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"], [\"top\",\"middle\",\"front\"]]"),
  ("Can you create a video showing the effect of a sunset, with the light dimming and changing color?", "[[\"top\",\"right\",\"front\"], [\"bottom\",\"left\",\"back\"]]"),
  ("I'd like a video simulating a walk through a city at night, with varying light sources from windows.", "[[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"], [\"top\",\"middle\",\"front\"]]"),
  ("Create a video that shows the flickering of a candle, with light moving subtly.", "[[\"bottom\",\"middle\",\"front\"], [\"bottom\",\"middle\",\"back\"], [\"bottom\",\"middle\",\"front\"]]"),
  ("Simulate a video of a flashlight being turned on in the dark, moving from front to top.", "[[\"middle\",\"middle\",\"front\"], [\"top\",\"middle\",\"front\"]]"),
  ("Show a video transition from indoor to outdoor lighting, from soft to harsh.", "[[\"middle\",\"middle\",\"front\"], [\"top\",\"middle\",\"middle\"]]"),
  ("I want a video that starts with a silhouette effect from backlighting, transitioning to full illumination.", "[[\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]]"),
  ("Can the video simulate an eclipse, with light dimming then brightening again?", "[[\"top\",\"middle\",\"front\"], [\"top\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]]"),
  ("Make a video with a sequence of lighting that mimics passing through different rooms, from bright to dim.", "[[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]]"),
  ("Create a video showing a gradual increase in light intensity, simulating sunrise.", "[[\"bottom\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]]"),
  ("Simulate a video with dynamic lighting, changing from soft daylight to a golden sunset.", "[[\"top\",\"middle\",\"front\"], [\"bottom\",\"right\",\"front\"]]"),
  ("I'd like a video that shows shadows growing longer as the light moves from top to bottom.", "[[\"top\",\"middle\",\"middle\"], [\"bottom\",\"middle\",\"front\"]]"),
  ("Can you make a video where the light source orbits around the subject, creating dynamic shadows?", "[[\"middle\",\"right\",\"front\"], [\"top\",\"middle\",\"front\"], [\"middle\",\"left\",\"front\"]]"),
  ("Show a video that simulates the flashing of stage lights, with rapid changes in direction and intensity.", "[[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"], [\"top\",\"middle\",\"front\"]]"),
  ("Create a video effect of a campfire night, with light flickering from the bottom and shadows dancing.", "[[\"bottom\",\"middle\",\"front\"], [\"bottom\",\"middle\",\"back\"], [\"bottom\",\"middle\",\"front\"]]"),
  ("Simulate a video of driving at night, with headlight beams moving from front to back.", "[[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]]"),
  ("I want a video showing a light pulse effect, with waves of light coming from the center.", "[[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]]"),
  ("Create a video where the lighting mimics a cloudy day, with brief moments of sunlight.", "[[\"top\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"], [\"top\",\"middle\",\"back\"]]"),
  ("Can the video have a sequence of a spotlight moving across a dark stage, focusing on different areas?", "[[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"]]"),
  ("Simulate a video of a sunrise to sunset cycle, with lighting changing accordingly.", "[[\"bottom\",\"middle\",\"front\"], [\"top\",\"middle\",\"middle\"], [\"bottom\",\"middle\",\"back\"]]"),
  ("Make a video that showcases a transition from artificial to natural lighting.", "[[\"middle\",\"middle\",\"front\"], [\"top\",\"middle\",\"front\"]]"),
  ("I'd like a video with a dramatic reveal effect, starting from darkness to a bright spotlight.", "[[\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]]"),
  ("Can you create a video showing a gradual dimming of light, as if the day is ending?", "[[\"top\",\"middle\",\"front\"], [\"bottom\",\"middle\",\"back\"]]"),
  ("Show a video that simulates moving shadows as clouds pass in front of the sun.", "[[\"top\",\"middle\",\"front\"], [\"top\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]]"),
  ("Direct the light to come from the left as if it's early morning.", "[\"middle\",\"left\",\"front\"]"),
  ("Can you make the lighting look like it's coming from directly behind the subject?", "[\"middle\",\"middle\",\"back\"]"),
  ("I want the light to simulate an overhead noon sun.", "[\"top\",\"middle\",\"middle\"]"),
  ("Adjust the lighting to appear as if it's coming from the bottom right, like a setting sun.", "[\"bottom\",\"right\",\"front\"]"),
  ("Please make the light source come from the top left, casting dramatic shadows.", "[\"top\",\"left\",\"middle\"]"),
  ("Simulate the lighting coming from the right side, mimicking an afternoon light.", "[\"middle\",\"right\",\"front\"]"),
  ("Could you create a soft light effect from the front as if it's dusk?", "[\"middle\",\"middle\",\"front\"]"),
  ("Make the lighting appear as if it's emanating from the bottom left corner.", "[\"bottom\",\"left\",\"middle\"]"),
  ("I'd like the light to be as if it's coming from behind and above, like a halo effect.", "[\"top\",\"middle\",\"back\"]"),
  ("Can the light be positioned to simulate a street light from the right at night?", "[\"middle\",\"right\",\"front\"]"),
  ("Adjust the light to mimic the effect of early dawn light from the bottom.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Create a shadow effect with the light coming from the top right corner.", "[\"top\",\"right\",\"middle\"]"),
  ("I want a backlight effect, as if the light source is directly behind the object.", "[\"middle\",\"middle\",\"back\"]"),
  ("Simulate a warm evening glow coming from the left side.", "[\"middle\",\"left\",\"front\"]"),
  ("Can the lighting give the impression of a bright, sunny day from above?", "[\"top\",\"middle\",\"middle\"]"),
  ("Make the scene look as if it's illuminated by a lamp from the lower right.", "[\"bottom\",\"right\",\"front\"]"),
  ("I'd like the light to come from the top, as if shining down from a skylight.", "[\"top\",\"middle\",\"middle\"]"),
  ("Can you create the effect of moonlight coming from the top left?", "[\"top\",\"left\",\"middle\"]"),
  ("Adjust the lighting to look like it's coming from a candle on the table, bottom center.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Simulate the light source as if it's a soft glow from the front right.", "[\"middle\",\"right\",\"front\"]"),
  ("Can the light appear as if it's reflecting off water from the bottom?", "[\"bottom\",\"middle\",\"front\"]"),
  ("I want the lighting to mimic a sunset, coming from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("Make the light seem as if it's coming from a window on the left side.", "[\"middle\",\"left\",\"front\"]"),
  ("Adjust the light to have a diffuse glow from above, like an overcast day.", "[\"top\",\"middle\",\"back\"]"),
  ("Simulate the lighting to be as if it's noon, with the sun directly overhead.", "[\"top\",\"middle\",\"middle\"]"),
  ("Can the light be set to mimic the golden hour, coming from the right?", "[\"middle\",\"right\",\"front\"]"),
  ("I'd like a dramatic lighting effect from the back left, creating long shadows.", "[\"middle\",\"left\",\"back\"]"),
  ("Create an effect of indirect light coming from the front, as if diffused through curtains.", "[\"middle\",\"middle\",\"front\"]"),
  ("Simulate a strong light source from the right, as if from a spotlight.", "[\"middle\",\"right\",\"front\"]"),
  ("Adjust the lighting to resemble a cozy indoor setting, coming from the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Make the lighting softer, as if it's a cloudy day, coming from above.", "[\"top\",\"middle\",\"back\"]"),
  ("Can the light source be positioned to mimic the effect of twilight from the left side?", "[\"middle\",\"left\",\"front\"]"),
  ("I want the scene to have a dramatic shadow effect with light from the top right.", "[\"top\",\"right\",\"middle\"]"),
  ("Simulate a bright light coming from the bottom center, like a stage spotlight.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Adjust the lighting to appear as if it's a dim, evening light from the right.", "[\"middle\",\"right\",\"back\"]"),
  ("Please create a morning light effect coming from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("Can you make the light look like it's coming from a high window on the left?", "[\"top\",\"left\",\"front\"]"),
  ("Mimic the effect of a lamp light from the right side at table height.", "[\"middle\",\"right\",\"front\"]"),
  ("Create a backlighting effect from the top, as if the sun is directly behind the subject.", "[\"top\",\"middle\",\"back\"]"),
  ("Simulate the ambiance of a soft overhead light, like in a gallery.", "[\"top\",\"middle\",\"middle\"]"),
  ("I'd like the lighting to give the impression of dusk, coming from the bottom right.", "[\"bottom\",\"right\",\"back\"]"),
  ("Make the light source seem as if it's coming from the direct front, like a camera flash.", "[\"middle\",\"middle\",\"front\"]"),
  ("Adjust the light to mimic a soft glow from the left, as if reflecting off snow.", "[\"middle\",\"left\",\"front\"]"),
  ("Simulate a strong midday sun coming from above and slightly to the right.", "[\"top\",\"right\",\"middle\"]"),
  ("Please have the lighting come from the back right, as if late afternoon sun.", "[\"middle\",\"right\",\"back\"]"),
  ("Can the light source appear to be from the upper front, like a spotlight on a stage?", "[\"top\",\"middle\",\"front\"]"),
  ("Mimic the lighting effect of a candle on the table, directly from the bottom.", "[\"bottom\",\"middle\",\"front\"]"),
  ("I want a shadow cast from the left, as if from a tall building in the afternoon.", "[\"middle\",\"left\",\"back\"]"),
  ("Create an effect where the light seems to be diffusing through a curtain from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Simulate a late evening light coming from the bottom left, like a sunset glow.", "[\"bottom\",\"left\",\"front\"]"),
  ("Adjust the lighting to look like early morning light, soft and from the top left.", "[\"top\",\"left\",\"front\"]"),
  ("Can the lighting give the feel of a cozy indoor lamp from the right side?", "[\"middle\",\"right\",\"front\"]"),
  ("Make the scene appear as if lit by a full moon from the top.", "[\"top\",\"middle\",\"middle\"]"),
  ("I'd like a spotlight effect from the bottom right, as if in a dramatic scene.", "[\"bottom\",\"right\",\"front\"]"),
  ("Simulate the soft lighting of an overcast day, coming from the top middle.", "[\"top\",\"middle\",\"back\"]"),
  ("Can we have the light appear as if coming through a dense forest from above?", "[\"top\",\"middle\",\"front\"]"),
  ("Mimic the effect of a strong sidelight from the left, as if in a painting.", "[\"middle\",\"left\",\"front\"]"),
  ("Please create a sharp shadow effect with light from the direct back.", "[\"middle\",\"middle\",\"back\"]"),
  ("Adjust the lighting to have a golden hour glow from the right side.", "[\"middle\",\"right\",\"front\"]"),
  ("Simulate a dimly lit scene with light coming from the front, as if from a distant light source.", "[\"middle\",\"middle\",\"front\"]"),
  ("Ensure the light mimics a bright summer noon from directly above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate a cozy fireplace glow from the bottom right corner.", "[\"bottom\",\"right\",\"front\"]"),
  ("Create the effect of a soft morning light from the top left.", "[\"top\",\"left\",\"front\"]"),
  ("Can the light cast a long shadow as if it's late afternoon from the right?", "[\"middle\",\"right\",\"back\"]"),
  ("Mimic an industrial overhead light, coming from directly above.", "[\"top\",\"middle\",\"middle\"]"),
  ("I'd like an evening shadow effect, with light from the bottom left.", "[\"bottom\",\"left\",\"back\"]"),
  ("Make the lighting appear as if it's a dimly lit streetlamp from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Adjust the light to simulate moonlight from the top right corner.", "[\"top\",\"right\",\"middle\"]"),
  ("Simulate a soft backlight from the bottom, creating a silhouette effect.", "[\"bottom\",\"middle\",\"back\"]"),
  ("Can we have a dramatic spotlight effect from the front left?", "[\"middle\",\"left\",\"front\"]"),
  ("Create a natural daylight effect, as if coming through a window from the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Mimic the golden glow of sunset coming from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("Simulate a bright studio light from the front, evenly illuminating the scene.", "[\"middle\",\"middle\",\"front\"]"),
  ("I want a mysterious effect with a dim light source from the back right.", "[\"middle\",\"right\",\"back\"]"),
  ("Adjust the lighting to give the impression of a cloudy day from above.", "[\"top\",\"middle\",\"back\"]"),
  ("Make the light feel like early dawn, soft and from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("Can the light source be as if it's a strong afternoon sun from the right?", "[\"middle\",\"right\",\"front\"]"),
  ("Simulate the effect of an overcast sky, with diffused light from the top.", "[\"top\",\"middle\",\"back\"]"),
  ("Create a warm evening ambiance with light from the top right.", "[\"top\",\"right\",\"front\"]"),
  ("Mimic a soft glow from the left, as if from a lamp on a table.", "[\"middle\",\"left\",\"front\"]"),
  ("I'd like a high contrast effect with sharp shadows from the bottom right.", "[\"bottom\",\"right\",\"middle\"]"),
  ("Can we have light as if it's reflecting off water, coming from the bottom?", "[\"bottom\",\"middle\",\"front\"]"),
  ("Simulate a theatrical spotlight from above, directly on the subject.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create an effect of twilight, with the light fading from the top left.", "[\"top\",\"left\",\"back\"]"),
  ("Make the scene look as if it's lit by candlelight from the center bottom.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Adjust for a soft, diffused light as if it's a foggy morning from the top.", "[\"top\",\"middle\",\"back\"]"),
  ("I want the lighting to mimic the harshness of a desert at noon from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Can the light source appear to be from an early morning sun from the left?", "[\"middle\",\"left\",\"front\"]"),
  ("Simulate a side light as if in a forest, coming from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Create a gentle evening glow from the bottom left, as if at sunset.", "[\"bottom\",\"left\",\"front\"]"),
  ("Mimic the effect of a bright light from a window on the right side.", "[\"middle\",\"right\",\"front\"]"),
  ("Adjust the light to simulate a clear, sunny day from directly above.", "[\"top\",\"middle\",\"middle\"]"),
  ("I'd like a shadowy effect with the light source coming from the back.", "[\"middle\",\"middle\",\"back\"]"),
  ("Can the scene be illuminated as if by a full moon from the top right?", "[\"top\",\"right\",\"middle\"]"),
  ("Simulate ambient light as if it's midday and overcast from above.", "[\"top\",\"middle\",\"back\"]"),
  ("Create a soft light effect from the front, as if it's a cloudy day.", "[\"middle\",\"middle\",\"front\"]"),
  ("Mimic a late afternoon sun casting long shadows from the right.", "[\"middle\",\"right\",\"back\"]"),
  ("I want a bright, direct light from the front, like a flash.", "[\"middle\",\"middle\",\"front\"]"),
  ("Can we have a backlit effect from the top left, creating silhouettes?", "[\"top\",\"left\",\"back\"]"),
  ("Simulate a warm indoor light coming from the left side.", "[\"middle\",\"left\",\"front\"]"),
  ("Create a dramatic effect with a strong backlight from the bottom.", "[\"bottom\",\"middle\",\"back\"]"),
  ("Adjust the lighting to mimic the softness of dusk from the right.", "[\"middle\",\"right\",\"back\"]"),
  ("Make the light source appear as if it's directly overhead at noon.", "[\"top\",\"middle\",\"middle\"]"),
  ("I'd like an effect as if the subject is under a spotlight from the top right.", "[\"top\",\"right\",\"front\"]"),
  ("Can the lighting simulate an early morning fog from the top left?", "[\"top\",\"left\",\"front\"]"),
  ("Simulate a scene lit by a single lamp from the right, casting soft shadows.", "[\"middle\",\"right\",\"front\"]"),
  ("Create a dynamic shadow effect with light coming from the top left.", "[\"top\",\"left\",\"middle\"]"),
  ("Mimic a soft, diffused morning light from the front and top.", "[\"top\",\"middle\",\"front\"]"),
  ("Adjust the scene to be lit as if by late afternoon sunlight from the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Mimic the serene light of early dawn coming from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("Create the illusion of a harsh midday shadow from the top.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate the gentle light of a sunset from the top left.", "[\"top\",\"left\",\"front\"]"),
  ("Adjust the scene to reflect a bright, indirect light from the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Can the lighting suggest a cool, blue tone from the right, like moonlight?", "[\"middle\",\"right\",\"front\"]"),
  ("Make it appear as if the light is filtering through trees from above.", "[\"top\",\"middle\",\"front\"]"),
  ("I want a stark contrast with a strong backlight from the bottom.", "[\"bottom\",\"middle\",\"back\"]"),
  ("Simulate a diffused, soft light from the front as if it's overcast.", "[\"middle\",\"middle\",\"front\"]"),
  ("Can we have the light seem as if it's coming from a single bulb overhead?", "[\"top\",\"middle\",\"middle\"]"),
  ("Mimic the warm glow of evening light coming from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("Create a dynamic effect as if the light moves from left to right across the subject.", "[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"]"),
  ("Adjust for a soft morning light coming from the top right.", "[\"top\",\"right\",\"front\"]"),
  ("Simulate a spotlight effect from the front, focusing on the subject.", "[\"middle\",\"middle\",\"front\"]"),
  ("I'd like a vintage effect, with light coming from the top left as if through a window.", "[\"top\",\"left\",\"front\"]"),
  ("Can the scene be lit as if by the golden hour sun from the right?", "[\"middle\",\"right\",\"front\"]"),
  ("Make the light source appear to be from a distant streetlight on the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Simulate the ambiance of a dimly lit room with light from the bottom.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Create the impression of noonday sun casting shadows from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Mimic the effect of a soft box light from the right, giving even illumination.", "[\"middle\",\"right\",\"front\"]"),
  ("I want a dramatic sidelight from the left, as if in a film noir scene.", "[\"middle\",\"left\",\"front\"]"),
  ("Can we have a backlit scenario with a slight glow from the top right?", "[\"top\",\"right\",\"back\"]"),
  ("Simulate a cold, early morning light from the top left, very soft.", "[\"top\",\"left\",\"front\"]"),
  ("Adjust the light to mimic a late afternoon sun, warm and from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Create a shadowy effect with minimal light from the bottom left.", "[\"bottom\",\"left\",\"back\"]"),
  ("Make it seem as if the subject is under harsh fluorescent light from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate the reflective glow of snow from the bottom, casting light upwards.", "[\"bottom\",\"middle\",\"front\"]"),
  ("I'd like an effect of dappled sunlight through leaves from the top right.", "[\"top\",\"right\",\"front\"]"),
  ("Can the lighting suggest a cozy indoor setting, with a lamp on the right?", "[\"middle\",\"right\",\"front\"]"),
  ("Mimic the clarity of a bright day with direct sunlight from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create an effect of twilight, with diffuse light from the left.", "[\"middle\",\"left\",\"back\"]"),
  ("Adjust for a beam of light from the right, as if through a crack in a door.", "[\"middle\",\"right\",\"front\"]"),
  ("Simulate the warm light of a candle from the center, glowing upwards.", "[\"bottom\",\"middle\",\"front\"]"),
  ("I want a backlit effect with intense light from behind, casting a silhouette.", "[\"middle\",\"middle\",\"back\"]"),
  ("Can we have a gentle morning light from the bottom right, softening the scene?", "[\"bottom\",\"right\",\"front\"]"),
  ("Mimic a setting sun with light coming from the bottom left, warm and inviting.", "[\"bottom\",\"left\",\"front\"]"),
  ("Create the illusion of artificial light from the left, like in a studio.", "[\"middle\",\"left\",\"front\"]"),
  ("Adjust the scene to be lit as if by a strong overhead light in the center.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate natural light coming from the front, as if on a cloudy day.", "[\"middle\",\"middle\",\"front\"]"),
  ("I'd like a subtle effect of dusk light from the top left, casting long shadows.", "[\"top\",\"left\",\"back\"]"),
  ("Can the lighting be as if from a bright headlight coming from the front right?", "[\"middle\",\"right\",\"front\"]"),
  ("Make the source of light appear as if from a high angle on the left, like afternoon sun.", "[\"top\",\"left\",\"front\"]"),
  ("Simulate a soft, indirect light from the right, as if diffused through curtains.", "[\"middle\",\"right\",\"front\"]"),
  ("Create a stark effect with a single light source from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("I want an eerie effect with a dim light from the back left, barely illuminating the scene.", "[\"middle\",\"left\",\"back\"]"),
  ("Can we have a vibrant sunrise effect from the bottom left, casting a warm glow?", "[\"bottom\",\"left\",\"front\"]"),
  ("Mimic the bright, direct light of a flash from the front, highlighting the subject.", "[\"middle\",\"middle\",\"front\"]"),
  ("Adjust for a romantic evening setting with light from the top right, soft and diffuse.", "[\"top\",\"right\",\"front\"]"),
  ("Simulate a harsh side light from the left, creating deep shadows.", "[\"middle\",\"left\",\"front\"]"),
  ("Create a dynamic effect with light as if moving through a forest, from top to bottom.", "[\"top\",\"middle\",\"front\"], [\"bottom\",\"middle\",\"front\"]"),
  ("Mimic the serene light of early dawn coming from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("Create the illusion of a harsh midday shadow from the top.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate the gentle light of a sunset from the top left.", "[\"top\",\"left\",\"front\"]"),
  ("Adjust the scene to reflect a bright, indirect light from the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Can the lighting suggest a cool, blue tone from the right, like moonlight?", "[\"middle\",\"right\",\"front\"]"),
  ("Make it appear as if the light is filtering through trees from above.", "[\"top\",\"middle\",\"front\"]"),
  ("I want a stark contrast with a strong backlight from the bottom.", "[\"bottom\",\"middle\",\"back\"]"),
  ("Simulate a diffused, soft light from the front as if it's overcast.", "[\"middle\",\"middle\",\"front\"]"),
  ("Can we have the light seem as if it's coming from a single bulb overhead?", "[\"top\",\"middle\",\"middle\"]"),
  ("Mimic the warm glow of evening light coming from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("Create a dynamic effect as if the light moves from left to right across the subject.", "[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"]"),
  ("Adjust for a soft morning light coming from the top right.", "[\"top\",\"right\",\"front\"]"),
  ("Simulate a spotlight effect from the front, focusing on the subject.", "[\"middle\",\"middle\",\"front\"]"),
  ("I'd like a vintage effect, with light coming from the top left as if through a window.", "[\"top\",\"left\",\"front\"]"),
  ("Can the scene be lit as if by the golden hour sun from the right?", "[\"middle\",\"right\",\"front\"]"),
  ("Make the light source appear to be from a distant streetlight on the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Simulate the ambiance of a dimly lit room with light from the bottom.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Create the impression of noonday sun casting shadows from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Mimic the effect of a soft box light from the right, giving even illumination.", "[\"middle\",\"right\",\"front\"]"),
  ("I want a dramatic sidelight from the left, as if in a film noir scene.", "[\"middle\",\"left\",\"front\"]"),
  ("Can we have a backlit scenario with a slight glow from the top right?", "[\"top\",\"right\",\"back\"]"),
  ("Simulate a cold, early morning light from the top left, very soft.", "[\"top\",\"left\",\"front\"]"),
  ("Adjust the light to mimic a late afternoon sun, warm and from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Create a shadowy effect with minimal light from the bottom left.", "[\"bottom\",\"left\",\"back\"]"),
  ("Make it seem as if the subject is under harsh fluorescent light from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate the reflective glow of snow from the bottom, casting light upwards.", "[\"bottom\",\"middle\",\"front\"]"),
  ("I'd like an effect of dappled sunlight through leaves from the top right.", "[\"top\",\"right\",\"front\"]"),
  ("Can the lighting suggest a cozy indoor setting, with a lamp on the right?", "[\"middle\",\"right\",\"front\"]"),
  ("Mimic the clarity of a bright day with direct sunlight from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create an effect of twilight, with diffuse light from the left.", "[\"middle\",\"left\",\"back\"]"),
  ("Adjust for a beam of light from the right, as if through a crack in a door.", "[\"middle\",\"right\",\"front\"]"),
  ("Simulate the warm light of a candle from the center, glowing upwards.", "[\"bottom\",\"middle\",\"front\"]"),
  ("I want a backlit effect with intense light from behind, casting a silhouette.", "[\"middle\",\"middle\",\"back\"]"),
  ("Can we have a gentle morning light from the bottom right, softening the scene?", "[\"bottom\",\"right\",\"front\"]"),
  ("Mimic a setting sun with light coming from the bottom left, warm and inviting.", "[\"bottom\",\"left\",\"front\"]"),
  ("Create the illusion of artificial light from the left, like in a studio.", "[\"middle\",\"left\",\"front\"]"),
  ("Adjust the scene to be lit as if by a strong overhead light in the center.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate natural light coming from the front, as if on a cloudy day.", "[\"middle\",\"middle\",\"front\"]"),
  ("I'd like a subtle effect of dusk light from the top left, casting long shadows.", "[\"top\",\"left\",\"back\"]"),
  ("Can the lighting be as if from a bright headlight coming from the front right?", "[\"middle\",\"right\",\"front\"]"),
  ("Make the source of light appear as if from a high angle on the left, like afternoon sun.", "[\"top\",\"left\",\"front\"]"),
  ("Simulate a soft, indirect light from the right, as if diffused through curtains.", "[\"middle\",\"right\",\"front\"]"),
  ("Create a stark effect with a single light source from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("I want an eerie effect with a dim light from the back left, barely illuminating the scene.", "[\"middle\",\"left\",\"back\"]"),
  ("Can we have a vibrant sunrise effect from the bottom left, casting a warm glow?", "[\"bottom\",\"left\",\"front\"]"),
  ("Mimic the bright, direct light of a flash from the front, highlighting the subject.", "[\"middle\",\"middle\",\"front\"]"),
  ("Adjust for a romantic evening setting with light from the top right, soft and diffuse.", "[\"top\",\"right\",\"front\"]"),
  ("Simulate a harsh side light from the left, creating deep shadows.", "[\"middle\",\"left\",\"front\"]"),
  ("Create a dynamic effect with light as if moving through a forest, from top to bottom.", "[\"top\",\"middle\",\"front\"], [\"bottom\",\"middle\",\"front\"]"),
  ("Create a soft dawn light coming from the left, not too harsh.", "[\"middle\",\"left\",\"middle\"]"),
  ("Simulate overhead noon lighting, casting minimal shadows.", "[\"top\",\"middle\",\"middle\"]"),
  ("I want an effect of a setting sun from the right, with a warm glow.", "[\"middle\",\"right\",\"front\"]"),
  ("Show the light as if it's reflecting off water from below.", "[\"bottom\",\"middle\",\"middle\"]"),
  ("Can you make it look like moonlight is coming from the top right?", "[\"top\",\"right\",\"middle\"]"),
  ("Mimic a cloudy day's diffused light coming uniformly from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Adjust the scene to have a soft lamp light from the right side.", "[\"middle\",\"right\",\"middle\"]"),
  ("Create a silhouette effect with backlighting from directly behind.", "[\"middle\",\"middle\",\"back\"]"),
  ("Simulate a side light effect as if from a window on the left.", "[\"middle\",\"left\",\"middle\"]"),
  ("I'd like a twilight ambiance with light from the bottom left.", "[\"bottom\",\"left\",\"middle\"]"),
  ("Can the light source be as if it's a bright LED from the front?", "[\"middle\",\"middle\",\"front\"]"),
  ("Make it appear as if early morning light is coming from the top left.", "[\"top\",\"left\",\"middle\"]"),
  ("Simulate a harsh midday sun casting shadows from directly overhead.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create a warm evening glow from the right, softening the scene.", "[\"middle\",\"right\",\"front\"]"),
  ("Mimic the effect of a spotlight on the subject from the front.", "[\"middle\",\"middle\",\"front\"]"),
  ("Adjust for a fading light of dusk coming from the top right.", "[\"top\",\"right\",\"middle\"]"),
  ("I want a cool, indirect light from the left, as if shaded.", "[\"middle\",\"left\",\"middle\"]"),
  ("Can we have a soft glow from the bottom, like a campfire?", "[\"bottom\",\"middle\",\"middle\"]"),
  ("Simulate the bright, unforgiving light of a desert at noon from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create a cinematic look with a strong light from the back right.", "[\"middle\",\"right\",\"back\"]"),
  ("Show a gradual change from daylight to sunset from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Mimic a soft, diffused morning light coming evenly from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("I'd like an artistic effect with dramatic shadows from the left.", "[\"middle\",\"left\",\"middle\"]"),
  ("Can the scene be under a harsh street light from above right?", "[\"top\",\"right\",\"middle\"]"),
  ("Make the subject appear as if under a full moon from the top.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate a cozy indoor feeling with light from the front left.", "[\"middle\",\"left\",\"front\"]"),
  ("Adjust the light as if it's late afternoon sun from the bottom right.", "[\"bottom\",\"right\",\"middle\"]"),
  ("Create an illusion of depth with light coming from the back.", "[\"middle\",\"middle\",\"back\"]"),
  ("I want a nostalgic feel with a dim light from the top left.", "[\"top\",\"left\",\"middle\"]"),
  ("Can you make it look like there's a soft haze from the front?", "[\"middle\",\"middle\",\"front\"]"),
  ("Simulate a bright workshop light coming down from directly above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create the impression of a cloudy, overcast day from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Mimic the glow of an early evening from the right, not too bright.", "[\"middle\",\"right\",\"middle\"]"),
  ("Adjust for a dramatic effect with a light from the bottom left.", "[\"bottom\",\"left\",\"middle\"]"),
  ("I want a sharp contrast with a strong light source from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Can we have a diffused light effect as if through thick fog from the top?", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate the ambient light of a bright room from all directions above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create a subdued light effect from the front, as if in a dimly lit room.", "[\"middle\",\"middle\",\"front\"]"),
  ("Mimic an autumn afternoon with light coming softly from the left.", "[\"middle\",\"left\",\"middle\"]"),
  ("Adjust the scene to have a golden hour warmth from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("I want a clear daylight effect with no direct shadows from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Can the light source mimic the effect of being underwater, from above?", "[\"top\",\"middle\",\"middle\"]"),
  ("Make it look like the light is bouncing off snow from the bottom.", "[\"bottom\",\"middle\",\"middle\"]"),
  ("Simulate a ring light effect directly from the front, evenly lit.", "[\"middle\",\"middle\",\"front\"]"),
  ("I'd like a high noon effect with minimal shadow from directly overhead.", "[\"top\",\"middle\",\"middle\"]"),
  ("Can there be a soft backlight creating a halo around the subject?", "[\"middle\",\"middle\",\"back\"]"),
  ("Mimic a late night scene with a dim light from the front right.", "[\"middle\",\"right\",\"front\"]"),
  ("Adjust for a morning light effect coming in from the top right.", "[\"top\",\"right\",\"middle\"]"),
  ("Create the look of a bright, sunny beach day from directly above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Shift the perspective to capture an early morning scene, with the light coming softly from the top left.", "[\"top\",\"left\",\"middle\"]"),
  ("Change the camera angle to look up, simulating sunlight filtering through trees from above.", "[\"top\",\"middle\",\"front\"]"),
  ("Adjust the viewpoint to mimic sunset, with the light warmly glowing from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("For a dramatic portrait, alter the angle to enhance shadows with light coming from the top right.", "[\"top\",\"right\",\"middle\"]"),
  ("Create a silhouette effect by changing the perspective to backlight from directly behind.", "[\"middle\",\"middle\",\"back\"]"),
  ("Simulate a cozy indoor atmosphere with a downward angle and soft light coming from the front right.", "[\"middle\",\"right\",\"front\"]"),
  ("Capture the feeling of dusk with an elevated view, having the light fade from the bottom left.", "[\"bottom\",\"left\",\"back\"]"),
  ("Enhance the depth of an urban night scene with light reflecting from the top, simulating street lamps.", "[\"top\",\"middle\",\"middle\"]"),
  ("Adjust for a macro shot with detailed texture, requiring direct light from the front to highlight details.", "[\"middle\",\"middle\",\"front\"]"),
  ("For an action scene, change the angle to catch dynamic shadows with light from the left.", "[\"middle\",\"left\",\"middle\"]"),
  ("Simulate an underwater view with diffused light coming from above, giving a muted, ethereal effect.", "[\"top\",\"middle\",\"middle\"]"),
  ("In a forest scene, alter the light to appear as if it's filtering through leaves from the top right.", "[\"top\",\"right\",\"front\"]"),
  ("Mimic an overcast day with a high perspective, requiring diffused light from all directions above.", "[\"top\",\"middle\",\"middle\"]"),
  ("For a vintage look, change the lighting to a soft glow from the bottom right, as if from an old lamp.", "[\"bottom\",\"right\",\"front\"]"),
  ("Create a stark industrial feel with harsh light coming from the top left, emphasizing metal textures.", "[\"top\",\"left\",\"middle\"]"),
  ("Adjust perspective to simulate evening light, with the low sun casting long shadows from the right.", "[\"middle\",\"right\",\"back\"]"),
  ("Simulate a sunrise with the perspective of light emerging from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("For a high fashion look, use direct overhead light to create bold contrasts and sharp shadows.", "[\"top\",\"middle\",\"middle\"]"),
  ("Capture a wintry scene with light coming from the front, mimicking a cloudy, snowy day.", "[\"middle\",\"middle\",\"front\"]"),
  ("Enhance a romantic mood by simulating candlelight from the center, giving a warm, intimate feel.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Create a cinematic night scene with a wide-angle perspective, light coming indirectly from the left.", "[\"middle\",\"left\",\"back\"]"),
  ("For a dramatic landscape, adjust the light to come from behind, casting the terrain in silhouette against the sky.", "[\"middle\",\"middle\",\"back\"]"),
  ("Simulate a studio portrait with balanced light from the front, ensuring even illumination of the subject.", "[\"middle\",\"middle\",\"front\"]"),
  ("In a street photography setting, mimic sunset light with a low angle coming from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Adjust to a bird's-eye view with the light source directly below, highlighting the subject from the ground.", "[\"bottom\",\"middle\",\"front\"]"),
   ("For an ethereal morning effect, angle the shot to catch the light diffusing softly from the top left.", "[\"top\",\"left\",\"middle\"]"),
  ("Create a high-contrast scene with the camera angled to capture sharp shadows from light on the right.", "[\"middle\",\"right\",\"middle\"]"),
  ("Simulate a late afternoon ambiance with the camera positioned to emphasize light from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("Enhance a sense of solitude in a vast landscape with the light source coming from the top, casting minimal shadows.", "[\"top\",\"middle\",\"middle\"]"),
  ("Capture the mystery of a foggy morning with a low angle shot and diffused light from the front.", "[\"middle\",\"middle\",\"front\"]"),
  ("For a dynamic sports action shot, position the camera to catch the dramatic shadows cast by light from the top right.", "[\"top\",\"right\",\"middle\"]"),
  ("Create an intimate indoor setting with a soft light source coming from the left, as if from a window.", "[\"middle\",\"left\",\"front\"]"),
  ("Simulate a moonlit scene with an upward angle, capturing the light from the top left as moonlight.", "[\"top\",\"left\",\"middle\"]"),
  ("For a vivid sunset portrait, adjust so the light appears to come from the bottom left, backlighting the subject.", "[\"bottom\",\"left\",\"back\"]"),
  ("Capture an urban evening with street lights from above, using a wide lens to get the light spread from the top.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate a bright summer day in the countryside with the camera angled to show light from directly overhead.", "[\"top\",\"middle\",\"middle\"]"),
  ("For an underwater perspective, mimic the light filtering down from the surface from the top middle.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create a nostalgic scene with the light coming from the right, casting long afternoon shadows.", "[\"middle\",\"right\",\"back\"]"),
  ("Emphasize a rugged landscape with harsh light from the top left, highlighting textures and forms.", "[\"top\",\"left\",\"middle\"]"),
  ("For a serene lake at dawn, position the light to mimic the first rays coming from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("Capture a bustling market scene with the light diffusing evenly from above, simulating midday bustle.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create a dramatic entrance with backlighting from the bottom, simulating spotlights on a stage.", "[\"bottom\",\"middle\",\"back\"]"),
  ("For a cozy winter evening, simulate a warm glow from the left, as if from a fireplace.", "[\"middle\",\"left\",\"front\"]"),
  ("Enhance the depth of a forest scene with light piercing through from the top right, simulating sunbeams.", "[\"top\",\"right\",\"front\"]"),
  ("Simulate a starry night with ambient light from above, giving a sense of vast open space.", "[\"top\",\"middle\",\"middle\"]"),
  ("For a studio fashion shoot, ensure the light comes directly from the front, creating a flattering effect.", "[\"middle\",\"middle\",\"front\"]"),
  ("Capture the solitude of a desert with the sun high and behind, creating minimal shadows from the top back.", "[\"top\",\"middle\",\"back\"]"),
  ("Create a sense of adventure with a low camera angle and the light coming from the top left, simulating early morning.", "[\"top\",\"left\",\"middle\"]"),
  ("Simulate an indoor workshop scene with focused light from above, highlighting the work area.", "[\"top\",\"middle\",\"middle\"]"),
  ("For a magical forest path, angle the shot to capture light streaming through trees from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Angle the shot to have morning light from the top left.", "[\"top\",\"left\",\"middle\"]"),
  ("Position camera for shadows from right side light.", "[\"middle\",\"right\",\"middle\"]"),
  ("Light the scene from below as if evening light from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("Direct light overhead for minimal shadow.", "[\"top\",\"middle\",\"middle\"]"),
  ("Front light for a foggy effect.", "[\"middle\",\"middle\",\"front\"]"),
  ("Capture action with top right lighting for dynamic shadows.", "[\"top\",\"right\",\"middle\"]"),
  ("Use left side window light for an indoor shot.", "[\"middle\",\"left\",\"front\"]"),
  ("Moonlight effect from top left.", "[\"top\",\"left\",\"middle\"]"),
  ("Backlight subject from the bottom left for silhouette.", "[\"bottom\",\"left\",\"back\"]"),
  ("Evening street light from above in a wide shot.", "[\"top\",\"middle\",\"middle\"]"),
  ("Bright noon light directly overhead in a countryside setting.", "[\"top\",\"middle\",\"middle\"]"),
  ("Light coming from top middle for underwater look.", "[\"top\",\"middle\",\"middle\"]"),
  ("Afternoon long shadows with right side light.", "[\"middle\",\"right\",\"back\"]"),
  ("Top left lighting for texture in a landscape.", "[\"top\",\"left\",\"middle\"]"),
  ("First light effect from the bottom right at dawn.", "[\"bottom\",\"right\",\"front\"]"),
  ("Midday light diffused from above in a market scene.", "[\"top\",\"middle\",\"middle\"]"),
  ("Spotlights from the bottom for a stage entrance.", "[\"bottom\",\"middle\",\"back\"]"),
  ("Fireplace glow from the left in an evening setting.", "[\"middle\",\"left\",\"front\"]"),
  ("Sunbeams through trees from the top right in a forest.", "[\"top\",\"right\",\"front\"]"),
  ("Ambient night sky light from above for starry effect.", "[\"top\",\"middle\",\"middle\"]"),
  ("Frontal lighting for a fashion shoot in studio.", "[\"middle\",\"middle\",\"front\"]"),
  ("Sun high and behind in a desert for minimal shadows.", "[\"top\",\"middle\",\"back\"]"),
  ("Early morning light from top left for adventure.", "[\"top\",\"left\",\"middle\"]"),
  ("Workshop scene with focused overhead light.", "[\"top\",\"middle\",\"middle\"]"),
  ("Light streaming through trees on the right for a path.", "[\"middle\",\"right\",\"front\"]"),
  ("Top left light for early morning effect.", "[\"top\",\"left\",\"middle\"]"),
  ("Side shadows with light from the right.", "[\"middle\",\"right\",\"middle\"]"),
  ("Evening ambiance with bottom right lighting.", "[\"bottom\",\"right\",\"front\"]"),
  ("Overhead light for midday outdoor scene.", "[\"top\",\"middle\",\"middle\"]"),
  ("Fog effect with frontal lighting.", "[\"middle\",\"middle\",\"front\"]"),
  ("Dynamic shadows from light at top right.", "[\"top\",\"right\",\"middle\"]"),
  ("Window light from the left for interior.", "[\"middle\",\"left\",\"front\"]"),
  ("Moonlight scene with top left illumination.", "[\"top\",\"left\",\"middle\"]"),
  ("Silhouette with backlight from bottom left.", "[\"bottom\",\"left\",\"back\"]"),
  ("Streetlamp light from above for night scene.", "[\"top\",\"middle\",\"middle\"]"),
  ("Noon sunlight directly from overhead.", "[\"top\",\"middle\",\"middle\"]"),
  ("Underwater effect with light from top middle.", "[\"top\",\"middle\",\"middle\"]"),
  ("Long shadows with afternoon light from right.", "[\"middle\",\"right\",\"back\"]"),
  ("Landscape texture highlighted by top left light.", "[\"top\",\"left\",\"middle\"]"),
  ("Dawn lighting effect from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("Diffused midday light from above for market.", "[\"top\",\"middle\",\"middle\"]"),
  ("Stage effect with spotlights from the bottom.", "[\"bottom\",\"middle\",\"back\"]"),
  ("Warm indoor lighting from the left side.", "[\"middle\",\"left\",\"front\"]"),
  ("Sunbeam effect in forest from top right.", "[\"top\",\"right\",\"front\"]"),
  ("Starry sky ambiance with overhead ambient light.", "[\"top\",\"middle\",\"middle\"]"),
  ("Front lighting for studio fashion photography.", "[\"middle\",\"middle\",\"front\"]"),
  ("Minimal shadow desert scene with backlight.", "[\"top\",\"middle\",\"back\"]"),
  ("Adventure scene lit from the top left.", "[\"top\",\"left\",\"middle\"]"),
  ("Focused light for workshop from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Pathway lit with right side light through trees.", "[\"middle\",\"right\",\"front\"]"),
  ("Change light angle to the top left corner.", "[\"top\",\"left\",\"middle\"]"),
  ("Position the camera so we see the light coming from the bottom right.", "[\"bottom\",\"right\",\"middle\"]"),
  ("Adjust light to shine directly from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Make the light source come from directly behind the subject.", "[\"middle\",\"middle\",\"back\"]"),
  ("Set the light to emulate early morning from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("Direct the light from the right to create evening shadows.", "[\"middle\",\"right\",\"back\"]"),
  ("Illuminate from the top as if it's noon.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate moonlight coming from the top right.", "[\"top\",\"right\",\"middle\"]"),
  ("Let the light flood in from the front, like a doorway opening.", "[\"middle\",\"middle\",\"front\"]"),
  ("Backlight the subject with light from the bottom.", "[\"bottom\",\"middle\",\"back\"]"),
  ("Highlight texture with light from the left side.", "[\"middle\",\"left\",\"middle\"]"),
  ("Illuminate the scene as if underwater, with light from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Create a sunset effect with light from the bottom right.", "[\"bottom\",\"right\",\"front\"]"),
  ("Cast long shadows with light coming from the top right.", "[\"top\",\"right\",\"middle\"]"),
  ("Mimic a cloudy day with diffuse light from the top.", "[\"top\",\"middle\",\"middle\"]"),
  ("Generate a soft glow effect from the bottom left.", "[\"bottom\",\"left\",\"front\"]"),
  ("Emphasize form with backlight from the top.", "[\"top\",\"middle\",\"back\"]"),
  ("Capture a night scene with ambient light from the middle.", "[\"middle\",\"middle\",\"middle\"]"),
  ("Show depth with a light source from the front right.", "[\"middle\",\"right\",\"front\"]"),
  ("Create a dramatic angle with light from the left, hitting the subject.", "[\"middle\",\"left\",\"middle\"]"),
   ("Angle the light from the top left to mimic morning light.", "[\"top\",\"left\",\"middle\"]"),
  ("Direct light from above for a noon effect.", "[\"top\",\"middle\",\"middle\"]"),
  ("Place the light behind the subject for a silhouette effect.", "[\"middle\",\"middle\",\"back\"]"),
  ("Illuminate from the bottom right to simulate evening light.", "[\"bottom\",\"right\",\"front\"]"),
  ("Set the lighting to come from the right side for shadows.", "[\"middle\",\"right\",\"middle\"]"),
  ("Use top right lighting for a late afternoon shadow.", "[\"top\",\"right\",\"middle\"]"),
  ("Position light from the front for a flat lighting effect.", "[\"middle\",\"middle\",\"front\"]"),
  ("Simulate moonlight from the top left corner.", "[\"top\",\"left\",\"middle\"]"),
  ("Create a sunset ambiance with bottom left lighting.", "[\"bottom\",\"left\",\"front\"]"),
  ("Emulate an overcast day with light from directly above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Generate a soft glow from the front right, like sunrise.", "[\"middle\",\"right\",\"front\"]"),
  ("Mimic a high noon sun with light from the top middle.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate twilight with light coming from the bottom.", "[\"bottom\",\"middle\",\"middle\"]"),
  ("Cast dramatic shadows with light from the left side.", "[\"middle\",\"left\",\"middle\"]"),
  ("Highlight textures with a direct light from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Produce a night scene effect with ambient light from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Simulate early dawn with bottom right light.", "[\"bottom\",\"right\",\"front\"]"),
  ("Illuminate the scene from the bottom left for a soft effect.", "[\"bottom\",\"left\",\"front\"]"),
  ("Show a high contrast effect with backlight from the top.", "[\"top\",\"middle\",\"back\"]"),
  ("Create a dynamic scene with light from the right for motion.", "[\"middle\",\"right\",\"middle\"]"),
  ("Mimic a studio look with frontal lighting.", "[\"middle\",\"middle\",\"front\"]"),
  ("Simulate indirect light from the left for a natural look.", "[\"middle\",\"left\",\"middle\"]"),
  ("Direct light from the bottom for an eerie effect.", "[\"bottom\",\"middle\",\"middle\"]"),
  ("Cast a warm glow from the top right, like sunset.", "[\"top\",\"right\",\"front\"]"),
  ("Use light from the top left to simulate a cloudy morning.", "[\"top\",\"left\",\"middle\"]"),
  ("Position the light to mimic afternoon sun from the right.", "[\"middle\",\"right\",\"front\"]"),
  ("Create a silhouette with light from directly behind.", "[\"middle\",\"middle\",\"back\"]"),
  ("Illuminate with a soft front light for an intimate setting.", "[\"middle\",\"middle\",\"front\"]"),
  ("Set a dramatic tone with sharp light from the left.", "[\"middle\",\"left\",\"middle\"]"),
  ("Simulate a streetlight effect with overhead lighting.", "[\"top\",\"middle\",\"middle\"]"),
  ("Mimic a candlelit scene with light from the bottom middle.", "[\"bottom\",\"middle\",\"front\"]"),
  ("Highlight with a spotlight from the front right.", "[\"middle\",\"right\",\"front\"]"),
  ("Generate a diffused lighting effect from the top for softness.", "[\"top\",\"middle\",\"middle\"]"),
  ("Show depth with angled light from the top right.", "[\"top\",\"right\",\"middle\"]"),
  ("Create a cozy atmosphere with bottom left lighting.", "[\"bottom\",\"left\",\"front\"]"),
  ("Simulate a clear day with bright light from above.", "[\"top\",\"middle\",\"middle\"]"),
  ("Emulate a sunset from the bottom right corner.", "[\"bottom\",\"right\",\"front\"]"),
  ("Achieve a mysterious mood with backlighting.", "[\"middle\",\"middle\",\"back\"]"),
  ("Illuminate for a morning look with light from the left.", "[\"middle\",\"left\",\"middle\"]"),
  ("Set up for an evening effect with light from the bottom.", "[\"bottom\",\"middle\",\"middle\"]"),
  ("Mimic a bright studio with light from the front.", "[\"middle\",\"middle\",\"front\"]"),
  ("Create soft shadows with light from the right side.", "[\"middle\",\"right\",\"middle\"]"),
  ("Use a backlight from the top for a halo effect.", "[\"top\",\"middle\",\"back\"]"),
  ("Simulate an indoor scene with light from the left.", "[\"middle\",\"left\",\"front\"]"),
  ("Achieve a natural morning effect with top left lighting.", "[\"top\",\"left\",\"middle\"]"),
  ("Produce a soft evening ambiance with bottom right light.", "[\"bottom\",\"right\",\"front\"]"),
  ("Direct light overhead for a midday summer look.", "[\"top\",\"middle\",\"middle\"]"),
  ("Cast shadows for depth with light from the top right.", "[\"top\",\"right\",\"middle\"]"),
  ("Illuminate from behind for a strong silhouette against the light.", "[\"middle\",\"middle\",\"back\"]"),
  ("Emphasize form with side lighting from the left.", "[\"middle\",\"left\",\"middle\"]"),
  ("Simulate a day-to-night transition with lighting starting from top middle to bottom left.", "[\"top\",\"middle\",\"middle\"], [\"bottom\",\"left\",\"front\"]"),
  ("Capture the progression of a sunrise from the bottom right to directly above.", "[\"bottom\",\"right\",\"front\"], [\"top\",\"middle\",\"middle\"]"),
  ("Show a moving shadow effect, with light shifting from the left side to the right side.", "[\"middle\",\"left\",\"middle\"], [\"middle\",\"right\",\"middle\"]"),
  ("Mimic the effect of clouds passing over the sun, from diffused top lighting to direct top lighting.", "[\"top\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]"),
  ("Illustrate a car's headlights at night moving from front to back.", "[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]"),
  ("Simulate a flashlight search, starting from the bottom right and moving to the top left.", "[\"bottom\",\"right\",\"front\"], [\"top\",\"left\",\"middle\"]"),
  ("Capture a flickering candle effect, alternating light from bottom middle to slightly higher.", "[\"bottom\",\"middle\",\"front\"], [\"middle\",\"middle\",\"front\"]"),
  ("Show a thunderstorm effect with intermittent lighting from above to the front.", "[\"top\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"], [\"top\",\"middle\",\"back\"]"),
  ("Mimic a moving sun from morning to noon, starting from the top left to directly above.", "[\"top\",\"left\",\"front\"], [\"top\",\"middle\",\"middle\"]"),
  ("Simulate stage lighting for a performance, moving from front right to back left.", "[\"middle\",\"right\",\"front\"], [\"middle\",\"left\",\"back\"]"),
  ("Illustrate a walk through a forest with light moving from top right to top left.", "[\"top\",\"right\",\"front\"], [\"top\",\"left\",\"front\"]"),
  ("Showcase a room brightening as the day progresses, from dim top left to bright top middle.", "[\"top\",\"left\",\"back\"], [\"top\",\"middle\",\"front\"]"),
  ("Capture the transition from indoor to outdoor lighting, starting from soft front to bright top.", "[\"middle\",\"middle\",\"front\"], [\"top\",\"middle\",\"middle\"]"),
  ("Mimic the light from a TV screen flickering, alternating rapidly from front to slightly dim.", "[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]"),
  ("Simulate a sunset to night transition, with light going from bottom right to top and fading.", "[\"bottom\",\"right\",\"front\"], [\"top\",\"middle\",\"back\"]"),
  ("Transition from dawn to full daylight, starting from the bottom left to top middle.", "[\"bottom\",\"left\",\"front\"], [\"top\",\"middle\",\"middle\"]"),
  ("Show the effect of walking under streetlights from front right to back left.", "[\"middle\",\"right\",\"front\"], [\"middle\",\"left\",\"back\"]"),
  ("Illustrate the moon rising, moving from the right side to the top right.", "[\"middle\",\"right\",\"middle\"], [\"top\",\"right\",\"middle\"]"),
  ("Capture the flicker of a campfire, with light alternating from bottom middle to bottom right.", "[\"bottom\",\"middle\",\"front\"], [\"bottom\",\"right\",\"front\"]"),
  ("Simulate a lighthouse beam, sweeping from left across to right and back.", "[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"], [\"middle\",\"left\",\"front\"]"),
  ("Demonstrate a cloud shadow moving across a landscape, from top middle to top left.", "[\"top\",\"middle\",\"middle\"], [\"top\",\"left\",\"back\"]"),
  ("Show a gradual room lighting from early morning to noon, bottom left to top.", "[\"bottom\",\"left\",\"front\"], [\"top\",\"middle\",\"middle\"]"),
  ("Mimic sunlight passing through a window, moving from left to right across the room.", "[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"]"),
  ("Illustrate a spotlight following a performer on stage, from front left to front right.", "[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"]"),
  ("Capture the dynamic shadows of clouds on a mountain, from top right to top left.", "[\"top\",\"right\",\"middle\"], [\"top\",\"left\",\"middle\"]"),
  ("Show the light change during an eclipse, from bright top middle to dim and back.", "[\"top\",\"middle\",\"front\"], [\"top\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]"),
  ("Simulate the shifting of car headlights in a tunnel, from front to back and front again.", "[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]"),
  ("Demonstrate a neon sign flickering on and off, alternating light from the front.", "[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]"),
  ("Illustrate the transition from artificial light to natural light in a room, from left to top.", "[\"middle\",\"left\",\"front\"], [\"top\",\"middle\",\"middle\"]"),
  ("Showcase the reflection of water waves on a ceiling, moving light from bottom to top.", "[\"bottom\",\"middle\",\"front\"], [\"top\",\"middle\",\"front\"]"),
  ("Capture a lightning storm with sudden flashes from above to front.", "[\"top\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]"),
  ("Mimic the effect of sunrise on a beach, from bottom right to top middle, then to top left.", "[\"bottom\",\"right\",\"front\"], [\"top\",\"middle\",\"middle\"], [\"top\",\"left\",\"middle\"]"),
  ("Illustrate changing stage lights during a concert, from bright front to colorful top.", "[\"middle\",\"middle\",\"front\"], [\"top\",\"middle\",\"front\"], [\"top\",\"left\",\"front\"]"),
  ("Show a day's end at a cafe, with sunlight from the right shifting to interior lights from the top.", "[\"middle\",\"right\",\"front\"], [\"top\",\"middle\",\"middle\"]"),
  ("Demonstrate a flashlight search across a dark room, from right to left then down.", "[\"middle\",\"right\",\"front\"], [\"middle\",\"left\",\"front\"], [\"bottom\",\"middle\",\"front\"]"),
  ("Capture the glow of a city at night from a high vantage, lights coming from below to the middle.", "[\"bottom\",\"middle\",\"front\"], [\"middle\",\"middle\",\"front\"]"),
  ("Mimic the opening of a door in a dark room, light from front increasing to full brightness.", "[\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]"),
  ("Illustrate a camera flash effect, from dim to bright front light and back to dim.", "[\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]"),
  ("Show the effect of a passing train at night, with light streaks moving from left to right.", "[\"middle\",\"left\",\"front\"], [\"middle\",\"right\",\"front\"]"),
  ("Simulate the rise and fall of stage lighting, from dim front to bright top, then dimming.", "[\"middle\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]"),
  ("Capture a sequence of a candle being blown out, from bright bottom to sudden darkness.", "[\"bottom\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"]"),
  ("Demonstrate the changing light within a forest from sunrise to full daylight, bottom to top.", "[\"bottom\",\"middle\",\"front\"], [\"top\",\"middle\",\"front\"]"),
  ("Show the transition of indoor lighting from evening to night, from soft right to overhead.", "[\"middle\",\"right\",\"front\"], [\"top\",\"middle\",\"middle\"]"),
  ("Mimic the effect of walking from shadow into sunlight, from dark front to bright top.", "[\"middle\",\"middle\",\"back\"], [\"top\",\"middle\",\"front\"]"),
  ("Illustrate the blinking of a neon sign, alternating rapidly from front to slightly dim and back.", "[\"middle\",\"middle\",\"front\"], [\"middle\",\"middle\",\"back\"], [\"middle\",\"middle\",\"front\"]")
                   ]


for sample in light_instruction:

    training_sequence = format_to_llama2_chat(
        "you are a helpful image assistant",
        [sample]
    )

    data_list.append({
        "text":training_sequence
    })

# for sample in legal_dataset['train']:

#     instruction_input_separator = random.choice([":", ": ", "\n", "\n\n", " "])
#     input = sample['input'] if sample['input'] is not None else ""
#     instruction = sample['instruction'] if sample['instruction'] is not None else ""

#     training_sequence = format_to_llama2_chat(
#         "you are a helpful legal assistant",
#         [(instruction+instruction_input_separator+input,sample['output'])]
#     )

#     data_list.append({
#         "text":training_sequence
#     })

print(len(data_list))
print(data_list[600])

621
{'text': '<s>[INST] <<SYS>> you are a helpful image assistant <</SYS>> Capture the dynamic shadows of clouds on a mountain, from top right to top left. [/INST] ["top","right","middle"], ["top","left","middle"] </s>'}


In [ ]:
data_list = []
for sample in legal_dataset['train']:

    instruction_input_separator = random.choice([":", ": ", "\n", "\n\n", " "])
    input = sample['input'] if sample['input'] is not None else ""
    instruction = sample['instruction'] if sample['instruction'] is not None else ""

    training_sequence = format_to_llama2_chat(
        "you are a helpful legal assistant",
        [(instruction+instruction_input_separator+input,sample['output'])]
    )

    data_list.append({
        "text":training_sequence
    })
print(len(data_list))
print(data_list[0])
together.Files.save_jsonl(data_list, "legal_dataset.jsonl")

4394
{'text': "<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.:Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST] The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Constitution. The court found that Rule 9(i) of the service rules, which allowed for termination of service without reas

In [ ]:
# save the reformatted dataset locally
together.Files.save_jsonl(data_list, "light_instruction.jsonl")

Wrote 621 records to light_instruction.jsonl


In [ ]:
# check your data with your base model prompting type before uploading
# resp = together.Files.check(file="legal_dataset.jsonl")
resp = together.Files.check(file="light_instruction.jsonl")
print(resp)

{'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.0 GB', 'num_samples': 621}


In [ ]:
resp = together.Files.list()
print(resp)

{'data': [{'filename': 'light_instruction.jsonl', 'bytes': 122959, 'created_at': 1708201421, 'id': 'file-0fba97e3-60f0-4344-8df3-864406d3e1dd', 'purpose': 'fine-tune', 'object': 'file', 'LineCount': 0, 'Processed': True}], 'object': 'list'}


In [ ]:
# upload your dataset file to together and save the file-id, youll need it to start your finetuning run
file_resp = together.Files.upload(file="light_instruction.jsonl")
file_id = file_resp["id"]
print(file_resp)

Uploading light_instruction.jsonl: 100%|██████████| 120k/120k [00:01<00:00, 93.3kB/s]

{'filename': 'light_instruction.jsonl', 'id': 'file-b79db231-affa-4662-ba78-fd981a737131', 'object': 'file', 'report_dict': {'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.0 GB', 'num_samples': 621}}


Expected output: ```{'filename': 'legal_dataset.jsonl', 'id': 'file-69649a68-6a36-41ad-8420-1750e99c26a7', 'object': 'file', 'report_dict': {'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File siz```

### Finetuning the base model on the new dataset to create the new finetuned model

depending on the size of the model and dataset this could take a few minutes to hours

In [ ]:
# Submit your finetune job
ft_resp = together.Finetune.create(
  training_file = file_id ,
  model = base_model_name,
  n_epochs = 20,
  batch_size = 4,
  n_checkpoints = 1,
  learning_rate = 5e-5,
  wandb_api_key = WANDB_API_KEY,
  #estimate_price = True,
  suffix = 'lighting',
)

fine_tune_id = ft_resp['id']
print(ft_resp)

{'training_file': 'file-b79db231-affa-4662-ba78-fd981a737131', 'validation_file': '', 'model_output_name': 'leelandzhang2026@u.northwestern.edu/Nous-Hermes-2-Mixtral-8x7B-DPO-lighting-2024-02-17-20-55-29', 'model_output_path': 's3://together-dev/finetune/65d0fa1e90ef1a37ebb8e62e/leelandzhang2026@u.northwestern.edu/Nous-Hermes-2-Mixtral-8x7B-DPO-lighting-2024-02-17-20-55-29/ft-e2c1bda9-3cc1-4185-a5ef-165165160b4a', 'Suffix': 'lighting', 'model': 'NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO', 'n_epochs': 20, 'n_checkpoints': 1, 'batch_size': 16, 'learning_rate': 5e-05, 'eval_steps': 0, 'user_id': '65d0fa1e90ef1a37ebb8e62e', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2024-02-17T20:55:29.645Z', 'updated_at': '2024-02-17T20:55:29.645Z', 'status': 'pending', 'owner_address': '0x08d0919b954c7bdabefc839e174de273d991d1ba', 'id': 'ft-e2c1bda9-3cc1-4185-a5ef-165165160b4a', 'job

Expected output:
```{'training_file': 'file-69649a68-6a36-41ad-8420-1750e99c26a7', 'model_output_name': 'carson/llama-2-7b-chat-law-2023-09-22-20-37-12', 'model_output_path': 's3://together-dev/finetune/64c4302a5cb247a0c80a3ddb/carson/llama-2-7b-chat-law-2023-09-22-20-37-12/ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf', 'Suffix': 'law', 'model': 'togethercomputer/llama-2-7b-chat', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 0.0001, 'user_id': '64c4302a5cb247a0c80a3ddb', 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-09-22T20:37:12.007Z', 'updated_at': '2023-09-22T20:37:12.007Z', 'status': 'pending', 'owner_address': '0xef5286fc0a1ac5bc4d4221cf3d51f1d97c45eaf7', 'id': 'ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf', 'job_id': '', 'token_count': 0, 'param_count': 0, 'total_price': 0, 'epochs_completed': 0, 'events': [{'object': 'fine-tune-event', 'created_at': '2023-09-22T20:37:12.007Z', 'level': '', 'message': 'Fine tune request created', 'type': 'JOB_PENDING', 'param_count': 0, 'token_count': 0, 'checkpoint_path': '', 'model_path': '', 'training_offset': 0, 'hash': ''}], 'queue_depth': 0, 'wandb_key': '', 'wandb_project_name': '', 'enable_checkpoints': False}```

In [ ]:
# run this from time to time to check on the status of your job
print(together.Finetune.retrieve(fine_tune_id=fine_tune_id)) # retrieves information on finetune event
print("-"*50)
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed
print(together.Finetune.is_final_model_available(fine_tune_id=fine_tune_id)) # True, False
print(together.Finetune.get_checkpoints(fine_tune_id=fine_tune_id)) # list of checkpoints

NameError: name 'together' is not defined

Monitor your training/finetuning progress using weights and biases. Below we verify that our model is learning and our training loss is decreasing.

<center><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/wandb.jpg" height=300 width=600> </center>


In [ ]:
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed

completed


get_job_status will transition from `pending`, `queued`, `running`, to `complete`.

when the job is finished, you should see:
```
{'training_file': 'file-69649a68-6a36-41ad-8420-1750e99c26a7', 'model_output_name': 'carson/llama-2-7b-chat-law-2023-09-22-20-37-12', 'model_output_path': 's3://together-dev/finetune/64c4302a5cb247a0c80a3ddb/carson/llama-2-7b-chat-law-2023-09-22-20-37-12/ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf-2023-09-22-13-47-25', 'Suffix': 'law', 'model': 'togethercomputer/llama-2-7b-chat', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 0.0001, 'user_id': '64c4302a5cb247a0c80a3ddb', 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-09-22T20:37:12.007Z', 'updated_at': '2023-09-22T20:52:52.675Z', 'status': 'completed', 'owner_address': '0xef5286fc0a1ac5bc4d4221cf3d51f1d97c45eaf7', 'id': 'ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf', 'job_id': '1509', 'token_count': 1676524, 'param_count': 6738415616, 'total_price': 5000000000, 'epochs_completed': 2, 'events': [{'object': 'fine-tune-event', 'created_at': '2023-09-22T20:37:12.007Z .....

completed

True
[]
```

The name of your finetuned model will show up in your list of models, but before you can start using it, you need to start it and it needs to finish deploying.

You can also find the name of your new model, start it and stop it, at https://api.together.xyz/playground

under `Models` > `My Model Instances`

<center><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/mymodels.jpg" height=300 width=600></center>

In [ ]:
# replace this name with the name of your newly finetuned model
new_model_name = 'leelandzhang@gmail.com/OpenHermes-2p5-Mistral-7B-lighting-2024-02-17-09-59-53'

model_list = together.Models.list()

print(f"{len(model_list)} models available")

available_model_names = [model_dict['name'] for model_dict in model_list]

new_model_name in available_model_names

164 models available


True

In [ ]:
# deploy your newly finetuned model
together.Models.start(new_model_name)

{'success': True,
 'value': 'b1d0e64eb80028e90687dc1701ea834cd8da2cec734c5720156866b39fe8bfc1-3bbeaf0d2671caa81aa289c6876b658905e9d104528257bd337ca5731f20e71e'}

In [ ]:
# check if your model is finished deploying, if this returns {"ready": true}, you model is ready for inference
together.Models.ready(new_model_name)

[{'modelInstanceConfig': {'appearsIn': [], 'order': 0},
  '_id': '65d087f2a40a7b4cc787eadd',
  'name': 'leelandzhang@gmail.com/OpenHermes-2p5-Mistral-7B-lighting-2024-02-17-09-59-53',
  'display_type': 'chat',
  'description': 'Continuation of OpenHermes 2 Mistral model trained on additional code datasets',
  'creator_organization': 'leelandzhang@gmail.com',
  'hardware_label': 'L40',
  'num_parameters': 7241732096,
  'release_date': '2024-02-17T10:18:26.668Z',
  'show_in_playground': True,
  'owner': 'leelandzhang@gmail.com',
  'owner_address': '0x11ff8254faa86ec5f5067a4c4da6b6a94dbf17f0',
  'owner_userid': '65d02832e27fee63c4251617',
  'parent': 'teknium/OpenHermes-2p5-Mistral-7B',
  'base': 'teknium/OpenHermes-2p5-Mistral-7B',
  'path': 'r2://together-dev/finetune/65d02832e27fee63c4251617/leelandzhang@gmail.com/OpenHermes-2p5-Mistral-7B-lighting-2024-02-17-09-59-53/ft-b13183f2-3ac1-4e68-be3e-6cc9b840a6b4-2024-02-17-02-13-58',
  'files': [{'filename': 'added_tokens.json',
    'size':

In [ ]:
# use the inference API to generate text / create completion / chat
print(new_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  model = new_model_name,
  max_tokens = 256,
  temperature = 0.5,
  top_k = 90,
  top_p = 0.8,
  repetition_penalty = 2.1,
  stop = ['</s>']
)

# print generated text
print(output['prompt'][0]+" -> "+output['output']['choices'][0]['text'])

leelandzhang@gmail.com/OpenHermes-2p5-Mistral-7B-lighting-2024-02-17-09-59-53
<s>[INST] <<SYS>> you are a helpful assistant <</SYS>> I want to change the lighting of the image so that it shines from left and above [/INST] -> frontrighttop light"] if ["leftbottom"," bottom["back top fronts right"],light],"just** back as ]]> with effect like ]",frontright day>>] coming </


In [ ]:
print("[\"top\", \"left\", \"middle\"]")

["top", "left", "middle"]


["top", "left", "middle"]

#### Example input separated from generated output by the ` -> ` sequence:


`<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.\n\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST]`

 ` -> `

 `The Supreme Court's decision in Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr. was based on several key legal principles and reasoning. Firstly, the court held that Article 14 of the Indian Constitution, which guarantees the right to equality, can apply to contractual terms between State entities and private parties if found arbitrary. This means that even though the employees were not government servants under the definition provided under Article 12, their service conditions could still be subject to constitutional scrutiny. Secondly, the court emphasized that Article 14 does not require proof of malice or mala fides for a termination to be deemed arbitrary. Instead, it is enough to show that the termination violates the principle of reasonableness and non-arbitrariness. Thirdly, the court rejected the doctrine of 'State instrumentality' as a valid defense against writ jurisdiction, stating that an entity will be considered 'State' only if there is deep and pervasive governmental control in its functioning and financing. Lastly, the court clarified that the right to livelihood under Article 21 is affected by arbitrary termination of employment. Therefore, .....`
  `</s>`

##### *the stop token is `</s>`*

In [ ]:
# stop your model and you will no longer be paying for it
together.Models.stop(new_model_name)

{'success': True}